In [21]:
import numpy as np 
import pandas as pd
import torch 
from torch import nn,optim
import torchtext
from torchtext import data

In [22]:
input_path="C:/Users/kusha/OneDrive/Desktop/Task/Deep_Learning_Everyday/semantics/"

In [ ]:
train_data_ac=pd.read_csv('C:/Users/kusha/OneDrive/Desktop/Task/Deep_Learning_Everyday/semantics/train.csv')
train_data_ac

In [ ]:
print('train_data.label', train_data_ac.label.value_counts())

In [ ]:
train_data_ac.head()

In [26]:
my_tokenizer=lambda x:str(x).split()

TEXT=data.Field(sequential=True,lower=True,tokenize=my_tokenizer,use_vocab=True)
LABEL=data.Field(sequential=False,use_vocab=False)

trainval_fields=[('text',TEXT),('label',LABEL)]

train_data,val_data=data.TabularDataset.splits(path=input_path, train='train.csv',validation='valid.csv',format='csv',skip_header=True,fields=trainval_fields)


In [ ]:
MAX_VOCAB_SIZE=25000
TEXT.build_vocab(train_data,max_size=MAX_VOCAB_SIZE)

In [31]:
train_iterator=data.BucketIterator(train_data,device='cpu',batch_size=32,sort_key=lambda x :len(x.text), sort_within_batch=False,repeat=False)


In [ ]:
val_iterator=data.BucketIterator(val_data,device='cpu',batch_size=32,sort_key= lambda x:len(x.text),sort_within_batch=False,repeat=False)


In [ ]:
print(TEXT.vocab.freqs.most_common()[:10]) 



In [ ]:
clemp='Hello World'
my_tokenizer = lambda x: str(x).split()
p=my_tokenizer(clemp)
print(p)

In [ ]:
my_tokenizer = lambda x: str(x).split()

# Example string
example_string = "This is a sample string."

# Tokenizing the string
tokens = my_tokenizer(example_string)
print(tokens)


In [44]:
class RNNModel(nn.Module):

    def __init__(self,embedding_dim,input_dim,hidden_dim,output_dim):
        super().__init__()
        self.Embedding=nn.Embedding(input_dim,embedding_dim)
        self.rnn=nn.RNN(embedding_dim,hidden_dim)
        self.fc=nn.Linear(hidden_dim,output_dim)
    
    def forward(self,text):
        embed=self.Embedding(text)
        output,hidden=self.rnn(embed)
        out=self.fc(hidden.squeeze(0))
        return out

In [45]:
input_dim=len(TEXT.vocab)
embedding_dim=100
hidden_dim=256
output_dim=1

model=RNNModel(embedding_dim,input_dim,hidden_dim,output_dim)

In [ ]:
def train(model,data_iterator,optimizer,loss_function):
    epoch_loss,epoch_acc,epoch_denom=0,0,0
    model.train()

    for i,batch in enumerate(data_iterator):
        optimizer.zero_grad()
        predictions=model(batch.text)
        loss=loss_function(predictions.reshape(-1,1),batch.label.float().reshape(-1,1))
        acc=accuracy(predictions.reshape(-1,1),batch.label.reshape(-1,1))
        loss.backward()
        optimizer.step()
        epoch_loss+=loss.item()
        epoch_acc+=acc.item()
        epoch_denom+=len(batch)
    return epoch_loss/epoch_denom,epoch_acc,epoch_denom




def accuracy(preds,y):
    rounded_preds=torch.round(torch.sigmoid(preds))
    correct=(rounded_preds)==y.float()
    acc=correct.sum()
    return acc


optimizer=torch.optim.Adam(model.parameters(),lr=1e-3)
loss_function=nn.BCEWithLogitsLoss()
n_epochs=5
for epoch in range(n_epochs):
    train_loss,train_acc,train_num=train(model,train_iterator,optimizer,loss_function)
    print('Epoch',epoch)
    print(train_loss)

In [1]:
# timeseries with RNN

import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt 
import matplotlib.pyplot as plt
import math

In [ ]:
start_date=dt.datetime(2015,4,1)
end_date=dt.datetime(2024,4,1)
data=yf.download('GOOGL',start_date,end_date)
data.head()

In [ ]:
len(data)

In [ ]:
training_data_len=math.ceil(len(data)*0.8)
training_data_len

In [5]:
train_data=data[:training_data_len].iloc[:,:1]

test_data=data[training_data_len:].iloc[:,:1]

In [ ]:
print(train_data.shape)
print(test_data.shape)

In [ ]:
train_data['Adj Close'].values

In [ ]:
test_data

In [8]:
dataset_train=train_data.values

In [9]:
dataset_train=dataset_train.reshape(-1,1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
scaled_train=scaler.fit_transform(dataset_train)
print(scaled_train)

In [ ]:
dataset_test = test_data.values 
# Reshaping 1D to 2D array
dataset_test = np.reshape(dataset_test, (-1,1))  
# Normalizing values between 0 and 1
scaled_test = scaler.transform(dataset_test)  
print(scaled_test[:5])

In [ ]:
scaled_train.shape

In [ ]:
scaled_test.shape

In [ ]:
scaled_test[50]

In [15]:
X_train=[]
y_train=[]

for i in range(50,len(scaled_train)):
    X_train.append(scaled_train[i-50:i,0])
    y_train.append(scaled_train[i,0])

    
    

In [ ]:
len(X_train)

In [ ]:
len(y_train)

In [18]:
X_test=[]
y_test=[]

for i in range(50,len(scaled_test)):
    X_test.append(scaled_test[i-50:i,0])
    y_test.append(scaled_test[i,0])

    
    

In [19]:
X_train,y_train=np.array(X_train),np.array(y_train)

In [ ]:
X_train.shape

In [21]:
y_train=np.reshape(y_train, (y_train.shape[0],1))

In [ ]:
y_train.shape

In [23]:
X_train=np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))

In [ ]:
X_train.shape

In [ ]:
# The data is converted to numpy array
X_test, y_test = np.array(X_test), np.array(y_test)

# #Reshaping
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))
y_test = np.reshape(y_test, (y_test.shape[0],1))
print("X_test :",X_test.shape,"y_test :",y_test.shape)

In [27]:
# modelling

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Dropout
from keras.layers import GRU,Bidirectional
from keras.optimizers import SGD
from sklearn import metrics
from sklearn.metrics import mean_squared_error




In [ ]:
regressor=Sequential()
regressor.add(SimpleRNN(units=50,activation='tanh',return_sequences=True,input_shape=(X_train.shape[1],1)))
regressor.add(Dropout(0.2))

regressor.add(SimpleRNN(units=50,activation='tanh',return_sequences=True))
regressor.add(SimpleRNN(units=50,activation='tanh',return_sequences=True))

regressor.add(SimpleRNN(units=50))

regressor.add(Dense(units=1,activation='sigmoid'))

regressor.compile(optimizer=SGD(learning_rate=0.01,decay=1e-6,momentum=0.9,nesterov=True),loss='mean_squared_error')

regressor.fit(X_train,y_train,epochs=20,batch_size=20)
regressor.summary()



In [ ]:
y_RNN=regressor.predict(X_test)
y_RNN_O=scaler.inverse_transform(y_RNN)


In [ ]:
fig, axs = plt.subplots(1,figsize =(18,12),sharex=True, sharey=True)
fig.suptitle('Model Predictions')

#Plot for RNN predictions
axs.plot(train_data.index[150:], train_data['Adj Close'][150:].values, label = "train_data", color = "b")
axs.plot(test_data.index[150:], test_data['Adj Close'][150:].values, label = "test_data", color = "g")
axs.plot(test_data.index[50:], y_RNN_O, label = "y_RNN", color = "brown")
axs.legend()
axs.title.set_text("Basic RNN")


plt.xlabel("Days")
plt.ylabel("Close price")

plt.show()


In [ ]:
train_data

In [ ]:
y_RNN_O

In [ ]:
train_data.index[150:] 

In [50]:
# improvised rnn

In [1]:
import numpy as np
import pandas as pd
import torch
import torchtext
from torch import nn, optim
from torch.optim import Adam
from torchtext import data

In [2]:
import spacy
from torchtext import data  

# Load the SpaCy model
spacy_en = spacy.load("en_core_web_sm")


def tokenize_spacy(text):
    return [token.text.lower() for token in spacy_en.tokenizer(text)]



TEXT = data.Field(sequential=True, tokenize=tokenize_spacy, include_lengths=True)
LABEL = data.Field(sequential=False, use_vocab=False)



In [3]:
spacy_en.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [4]:
input_path="C:/Users/kusha/OneDrive/Desktop/Task/Deep_Learning_Everyday/semantics/"

In [5]:
trainval_fields=[('text',TEXT),('label',LABEL)]

train_data,val_data=data.TabularDataset.splits(path=input_path,train='Train.csv',validation='Valid.csv',format='csv',skip_header=True,fields=trainval_fields)




MAX_VOCAB_SIZE=25000
TEXT.build_vocab(train_data,max_size=MAX_VOCAB_SIZE) 


In [7]:
BATCH_SIZE=64 

In [8]:
train_iterator=data.BucketIterator(train_data,batch_size=32,sort_key=lambda x : len(x.text),sort_within_batch=False,repeat=False)

val_iterator=data.BucketIterator(val_data,batch_size=32,sort_key=lambda x : len(x.text),sort_within_batch=False,repeat=False)


In [9]:
print(TEXT.vocab.freqs.most_common()[:10])

[('the', 521872), (',', 434690), ('.', 374841), ('and', 257963), ('a', 256539), ('of', 231186), ('to', 214108), ('is', 173431), ('it', 148293), ('in', 147249)]


In [10]:
class ImprovedRNN(nn.Module):
    def __init__(self,vocab_size,embedding_dim,hidden_dim,output_dim,n_layers,bidirectional,dropout,pad_idx):
        super().__init__()
        self.embedding=nn.Embedding(vocab_size,embedding_dim,padding_idx=pad_idx)
        self.lstm=nn.LSTM(embedding_dim,hidden_dim, num_layers=n_layers,bidirectional=bidirectional,dropout=dropout)
        self.fc=nn.Linear(hidden_dim*2,output_dim)
        self.dropout=nn.Dropout(dropout)

    def forward(self,text,text_lengths):
        embedded=self.dropout(self.embedding(text))
        packed_embedded=nn.utils.rnn.pack_padded_sequence(embedded,text_lengths)
        packed_output,(hidden,cell)=self.lstm(packed_embedded)
        output,output_lengths=nn.utils.rnn.pad_packed_sequence(packed_output)
        hidden=self.dropout(torch.cat((hidden[-2:,:],hidden[-1,:,:]),dim=1))
        return self.fc(hidden)
    
    
    
    



In [14]:

#importing libraries
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
 
x = {'processing', 'the', 'world', 'prime',
    'natural', 'language'}
 
# create the dictionary
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)
 
#this will print the dictionary of the words mapped with their indexes
print("Dictionary is = ", tokenizer.word_index)

Dictionary is =  {'processing': 1, 'natural': 2, 'world': 3, 'language': 4, 'the': 5, 'prime': 6}
